In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as mpathces
from matplotlib import animation
from time import time
%matplotlib qt

In [2]:
Number_of_particles = 100
Noise_amp = 0.1
SIZE = 0.005

ParticlesPerUnit = 3
RadiusSystem = int(np.sqrt(Number_of_particles / ParticlesPerUnit)) + 1
RatioOfInteraction = 1.
dt = 1./60
# dt = 1. / (10*np.power(Number_of_particles,0.5))

print('RadiusSystem = {}, Number_of_particles = {}, Density = {}, dt = {}'.format(RadiusSystem, Number_of_particles,
                                                                                 ParticlesPerUnit, dt))

#------------------------------Initial_State-------------------------

Distrib = np.random.poisson(1, Number_of_particles) + 1
Norm = np.amax(Distrib)
P = 20.*Distrib/Norm


degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = 0.1+0.9*np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()

Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State


class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[0, 0, 0, 0],
                 size=0.04,
                 noise=2 * np.pi,
                 RadiusSystem=1,
                 RatioOfInteraction=1):
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise
        self.noise_action = 0;
        self.size = size
        self.state = self.init_state.copy()
        self.state1 = self.init_state.copy()
        self.state_col = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt * np.ones(len(self.state[:, 0]))
        self.SQRT = np.ones(len(self.state[:, 0]))
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.mc = np.arange(3)
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 )
        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b // self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)


    def step(self, dt):

        self.time_elapsed += dt

        self.block[:] = -1
        self.state1 = self.state.copy()
        self.state_col = self.state.copy()
        self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
        self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
        self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))

        sx0 = 0
        sx1 = 0
        Vx = 0
        Vy = 0
        for i in range(len(self.state[:, 0])):
            self.lscl[i] = -1
            b = int(math.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * int(math.floor(np.abs(self.state[i, 1])))
            self.lscl[i] = self.block[b]
            self.block[b] = i

        for b0 in range(self.R2):
            f = int(self.block[b0])  
            while f != -1:
                for b1 in range(8):
                    b2 = int(self.cellb[b0][b1])
                    j = int(self.block[b2])  
                    while j != -1:
                        dist = np.sqrt((self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)
                        

                        if dist < self.RatioOfInteraction and f != j:
                            sx0 += self.state_orts[j, 2]
                            sx1 += self.state_orts[j, 3]

                        j = int(self.lscl[j])
                        
                self.noise_action = self.noise * (np.pi - 2  * np.random.rand() * np.pi)
                if (np.abs(sx0) < 0.0001) & (np.abs(sx1) < 0.0001):
                    
                    Vx = self.state_orts[f, 2]
                    Vy = self.state_orts[f, 3]

                    self.state_orts[f, 2] = Vx * np.cos(self.noise_action) - Vy * np.sin(self.noise_action)
                    self.state_orts[f, 3] = Vx * np.sin(self.noise_action) + Vy * np.cos(self.noise_action)
                else:
                    self.state_orts[f, 2] = (sx0 * np.cos(self.noise_action) - sx1 * np.sin(self.noise_action)) / np.hypot(sx0, sx1)
                    self.state_orts[f, 3] = (sx0 * np.sin(self.noise_action) + sx1 * np.cos(self.noise_action)) / np.hypot(sx0, sx1)

                sx0 = 0
                sx1 = 0
                self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                    self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2))
                    self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                    x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                    y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                    Vx = self.state[f, 2]
                    Vy = self.state[f, 3]

                    self.state[f, 3] = (Vy * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 * Vx * x01 * y01) / self.RadiusSystem ** 2
                    self.state[f, 2] = (Vx * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 * Vy * x01 * y01) / self.RadiusSystem ** 2

                    self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                    self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                    if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                        self.state[f, 0] = x01
                        self.state[f, 1] = y01
    
                else:
                    self.state[f, 0] += dt * self.state[f, 2]
                    self.state[f, 1] += dt * self.state[f, 3]
                            
                f = int(self.lscl[f])
                
box = ParticleBox(init_state, size=SIZE, noise=Noise_amp, RatioOfInteraction=RatioOfInteraction, RadiusSystem=RadiusSystem)            
plt.ion()
fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-RadiusSystem - 1, RadiusSystem + 1), ylim=(-RadiusSystem - 1, RadiusSystem + 1))

particles, = ax.plot([], [], 'bo', ms=1)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)


def init():
    global box
    particles.set_data([], [])
    time_text.set_text('')
    return particles, time_text


def animate(i):
    global box, dt, ax, fig
    box.step(dt)

    ms = int(fig.dpi * 5 * box.size)

    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles.set_markersize(ms)
    field = plt.quiver(box.state[:, 0], box.state[:, 1],box.state[:, 2], box.state[:, 3],edgecolor='k',width=0.01,scale=400)
    rw = plt.Circle((box.state[0, 0], box.state[0, 1]) , 1. ,fc='r',alpha=0.5)
    ax.add_patch(rw)
    time_text.set_text('time = %.3f' % box.time_elapsed)
    return particles, time_text, rw, field


t0 = time()
animate(0)
t1 = time()
interval = 1000 * dt - (t1 - t0)

ani = animation.FuncAnimation(fig, animate, frames=600,
                              interval=interval, blit=True, init_func=init)
circ = plt.Circle((0, 0), RadiusSystem + 0.1, fc='w')
ax.add_patch(circ)

# ani.save('particle_box_Viscek.wmv', fps=100, extra_args=['-vcodec', 'libx264'])
print("Done!")
plt.show()


RadiusSystem = 6, Number_of_particles = 100, Density = 3, dt = 0.016666666666666666
Done!


In [53]:
?plt.Circle

In [30]:
Number_of_particles = 100
Noise = 0.  # degree
SIZE = 0.005

ParticlesPerUnit = 70
RadiusSystem = int(np.sqrt(Number_of_particles / ParticlesPerUnit)) + 1
RatioOfInteraction = 1.
dt = 1. / 60
# dt = 1. / (10*np.power(Number_of_particles,0.5))

print('RadiusSystem = {}, Number_of_particles = {}, Density = {}, dt = {}'.format(RadiusSystem, Number_of_particles,
                                                                                 ParticlesPerUnit, dt))


P = np.random.poisson(1, Number_of_particles) + 4

degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = 0.1+0.9*np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()

Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State

box = ParticleBox(init_state, size=SIZE, noise=Noise, RatioOfInteraction=RatioOfInteraction, RadiusSystem=RadiusSystem)

class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[0, 0, 0, 0],
                 size=0.04,
                 noise=2 * np.pi,
                 RadiusSystem=1,
                 RatioOfInteraction=1):
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise * np.pi / 180
        self.noise_action = 0;
        self.size = size
        self.state = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt 
        self.SQRT = 0.
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.mc = np.arange(3)
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 )
        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b // self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)


    def step(self, dt):

        self.time_elapsed += dt

        self.block[:] = -1
        self.state_col = self.state.copy()
        self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
        self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
        self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))

        sx0 = 0
        sx1 = 0
        for i in range(len(self.state[:, 0])):
            self.lscl[i] = -1
            b = int(math.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * int(math.floor(np.abs(self.state[i, 1])))
            self.lscl[i] = self.block[b]
            self.block[b] = i

        for b0 in range(self.R2):
            f = int(self.block[b0])  
            while f != -1:
                for b1 in range(8):
                    b2 = int(self.cellb[b0][b1])
                    j = int(self.block[b2])  
                    while j != -1:
                        #dist = np.sqrt((self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)
                        

                        #if dist < self.RatioOfInteraction and f != j:
                        sx0 += self.state_orts[j, 2]
                        sx1 += self.state_orts[j, 3]

                        j = int(self.lscl[j])
                
                self.noise_action = self.noise * (np.pi - 2  * np.random.rand() * np.pi)
                if (np.abs(sx0) < 0.0001) & (np.abs(sx1) < 0.0001):
                    Vx = self.state_orts[f, 2]
                    Vy = self.state_orts[f, 3]

                    self.state_orts[f, 2] = Vx * np.cos(self.noise_action) - Vy * np.sin(self.noise_action)
                    self.state_orts[f, 3] = Vx * np.sin(self.noise_action) + Vy * np.cos(self.noise_action)
                else:
                    self.state_orts[f, 2] = (sx0 * np.cos(self.noise_action) - sx1 * np.sin(self.noise_action)) / np.hypot(sx0, sx1)
                    self.state_orts[f, 3] = (sx0 * np.sin(self.noise_action) + sx1 * np.cos(self.noise_action)) / np.hypot(sx0, sx1)

                sx0 = 0
                sx1 = 0
                self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                    self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2))
                    self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                    x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                    y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                    Vx = self.state[f, 2]
                    Vy = self.state[f, 3]

                    self.state[f, 3] = (Vy * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 * Vx * x01 * y01) / self.RadiusSystem ** 2
                    self.state[f, 2] = (Vx * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 * Vy * x01 * y01) / self.RadiusSystem ** 2

                    self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                    self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                    if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                        self.state[f, 0] = x01
                        self.state[f, 1] = y01
    
                else:
                    self.state[f, 0] += dt * self.state[f, 2]
                    self.state[f, 1] += dt * self.state[f, 3]
                            
                f = int(self.lscl[f])
                
            

fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-RadiusSystem - 1, RadiusSystem + 1), ylim=(-RadiusSystem - 1, RadiusSystem + 1))

particles, = ax.plot([], [], 'bo', ms=1)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

rect = plt.Rectangle(box.bounds[::2],
                     box.bounds[1] - box.bounds[0],
                     box.bounds[3] - box.bounds[2],
                     ec='none', lw=2, fc='none')
ax.add_patch(rect)


def init():
    global box, rect
    particles.set_data([], [])
    rect.set_edgecolor('none')
    time_text.set_text('')
    return particles, rect, time_text


def animate(i):
    global box, rect, dt, ax, fig
    box.step(dt)

    ms = int(fig.dpi * 5 * box.size)

    rect.set_edgecolor('w')
    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles.set_markersize(ms)
    field = plt.quiver(box.state[:, 0], box.state[:, 1],box.state[:, 2], box.state[:, 3],edgecolor='k',width=0.01,scale=400)
    time_text.set_text('time = %.3f' % box.time_elapsed)
    rw = plt.Circle((box.state[0, 0], box.state[0, 1]) , 1. ,fc='r',alpha=0.5)
    ax.add_patch(rw)
    return particles, rect, time_text, field, rw


t0 = time()
animate(0)
t1 = time()
interval = 1000 * dt - (t1 - t0)

ani = animation.FuncAnimation(fig, animate, frames=600,
                              interval=interval, blit=True, init_func=init)
circ = plt.Circle((0, 0), RadiusSystem + 0.1, fc='w')
ax.add_patch(circ)

# ani.save('particle_box_Viscek.wmv', fps=100, extra_args=['-vcodec', 'libx264'])
print("Done!")
plt.show()


RadiusSystem = 2, Number_of_particles = 100, Density = 70, dt = 0.016666666666666666
Done!


In [16]:
?ax.quiver

In [73]:
from __future__ import absolute_import, print_function
import numpy as np
import pyopencl as cl

In [77]:
a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
print(res_np - (a_np + b_np))
print(np.linalg.norm(res_np - (a_np + b_np)))

[ 0.  0.  0. ...,  0.  0.  0.]
0.0


In [78]:
import numpy as np
 
a = np.arange(32).astype(np.float32)
res = np.empty_like(a)
 
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
 
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
dest_buf = cl.Buffer(ctx, mf.WRITE_ONLY, res.nbytes)
 
prg = cl.Program(ctx, """
    __kernel void sq(__global const float *a,
    __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = a[gid] * a[gid];
    }
    """).build()
 
prg.sq(queue, a.shape, None, a_buf, dest_buf)
 
cl.enqueue_copy(queue, res, dest_buf)
 
print(a, res)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.] [   0.    1.    4.    9.   16.   25.   36.   49.   64.   81.  100.  121.
  144.  169.  196.  225.  256.  289.  324.  361.  400.  441.  484.  529.
  576.  625.  676.  729.  784.  841.  900.  961.]


# Vector noise 

In [3]:
import matplotlib
matplotlib.use('Qt4Agg')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import animation
from time import time


Number_of_particles = 100
Noise_amp = 0.25
SIZE = 0.005

ParticlesPerUnit = 3
RadiusSystem = int(np.sqrt(Number_of_particles / ParticlesPerUnit)) + 1
RatioOfInteraction = 1.
dt = 1./30
# dt = 1. / (10*np.power(Number_of_particles,0.5))

print('RadiusSystem = {}, Number_of_particles = {}, Density = {}, dt = {}'.format(RadiusSystem, Number_of_particles,
                                                                                 ParticlesPerUnit, dt))

#------------------------------Initial_State-------------------------

Distrib = np.random.poisson(1, Number_of_particles) + 1
Norm = np.amax(Distrib)
P = 10.*Distrib/Norm


degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = 0.1+0.9*np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()

Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State


class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[0, 0, 0, 0],
                 size=0.04,
                 noise=2 * np.pi,
                 RadiusSystem=1,
                 RatioOfInteraction=1):
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise
        self.noise_action = 0.;
        self.size = size
        self.state = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt
        self.SQRT = 0.
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.mc = np.arange(3)
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 )
        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b // self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)


    def step(self, dt):

        self.time_elapsed += dt

        self.block[:] = -1
        self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
        self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
        self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))

        sx0 = 0
        sx1 = 0
        Vx = 0
        Vy = 0
        for i in range(len(self.state[:, 0])):
            self.lscl[i] = -1
            b = int(math.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * int(math.floor(np.abs(self.state[i, 1])))
            self.lscl[i] = self.block[b]
            self.block[b] = i

        for b0 in range(self.R2):
            f = int(self.block[b0])  
            while f != -1:
                for b1 in range(8):
                    b2 = int(self.cellb[b0][b1])
                    j = int(self.block[b2])
                    n=0
                    while j != -1:
                        dist = np.sqrt((self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)
                        

                        if dist < self.RatioOfInteraction and f != j:
                            sx0 += self.state_orts[j, 2]
                            sx1 += self.state_orts[j, 3]
                            
                            n+=1
                        j = int(self.lscl[j])
                
                self.noise_action = self.noise * (np.pi - 2  * np.random.rand() * np.pi)
                if (np.abs(sx0) < 0.0001) & (np.abs(sx1) < 0.0001):
                    self.state_orts[f, 2] = self.state_orts[f, 2]
                    self.state_orts[f, 3] = self.state_orts[f, 3]
                else:
                    self.state_orts[f, 2] = (sx0 + self.noise*n*np.cos(self.noise_action)) / np.hypot(sx0 + self.noise*n*np.cos(self.noise_action), sx1 + self.noise*n*np.sin(self.noise_action))
                    self.state_orts[f, 3] = (sx1 + self.noise*n*np.sin(self.noise_action)) / np.hypot(sx0 + self.noise*n*np.cos(self.noise_action), sx1 + self.noise*n*np.sin(self.noise_action))

                sx0 = 0
                sx1 = 0
                n = 0
                self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                    self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2))
                    self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                    x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                    y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                    Vx = self.state[f, 2]
                    Vy = self.state[f, 3]

                    self.state[f, 3] = (Vy * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 * Vx * x01 * y01) / self.RadiusSystem ** 2
                    self.state[f, 2] = (Vx * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 * Vy * x01 * y01) / self.RadiusSystem ** 2

                    self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                    self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                    if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                        self.state[f, 0] = x01
                        self.state[f, 1] = y01
    
                else:
                    self.state[f, 0] += dt * self.state[f, 2]
                    self.state[f, 1] += dt * self.state[f, 3]
                            
                f = int(self.lscl[f])
                
box = ParticleBox(init_state, size=SIZE, noise=Noise_amp, RatioOfInteraction=RatioOfInteraction, RadiusSystem=RadiusSystem)            

fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-RadiusSystem - 1, RadiusSystem + 1), ylim=(-RadiusSystem - 1, RadiusSystem + 1))

particles, = ax.plot([], [], 'bo', ms=1)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)


def init():
    global box
    particles.set_data([], [])
    time_text.set_text('')
    return particles, time_text


def animate(i):
    global box, dt, ax, fig
    box.step(dt)

    ms = int(fig.dpi * 5 * box.size)

    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles.set_markersize(ms)
    field = plt.quiver(box.state[:, 0], box.state[:, 1],box.state[:, 2], box.state[:, 3],edgecolor='k',width=0.01,scale=400)
    rw = plt.Circle((box.state[0, 0], box.state[0, 1]) , 1. ,fc='r',alpha=0.5)
    ax.add_patch(rw)
    time_text.set_text('time = %.3f' % box.time_elapsed)
    return particles, time_text, rw, field


t0 = time()
animate(0)
t1 = time()
interval = 1000 * dt - (t1 - t0)

ani = animation.FuncAnimation(fig, animate, frames=600,
                              interval=interval, blit=True, init_func=init)
circ = plt.Circle((0, 0), RadiusSystem + 0.1, fc='w')
ax.add_patch(circ)

# ani.save('particle_box_Viscek.wmv', fps=100, extra_args=['-vcodec', 'libx264'])
print("Done!")
plt.show()

RadiusSystem = 6, Number_of_particles = 100, Density = 3, dt = 0.03333333333333333
Done!


In [43]:
Number_of_particles = 100
Noise = 0.  # degree
SIZE = 0.005

ParticlesPerUnit = 3
RadiusSystem = int(np.sqrt(Number_of_particles / ParticlesPerUnit)) + 1
RatioOfInteraction = 1.
dt = 1. / 30
# dt = 1. / (10*np.power(Number_of_particles,0.5))

print('RadiusSystem = {}, Number_of_particles = {}, Density = {}, dt = {}'.format(RadiusSystem, Number_of_particles,
                                                                                 ParticlesPerUnit, dt))


P = np.random.poisson(1, Number_of_particles) + 4

degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = 0.1+0.9*np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()

Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State

box = ParticleBox(init_state, size=SIZE, noise=Noise, RatioOfInteraction=RatioOfInteraction, RadiusSystem=RadiusSystem)

class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[0, 0, 0, 0],
                 size=0.04,
                 noise=2 * np.pi,
                 RadiusSystem=1,
                 RatioOfInteraction=1):
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise * np.pi / 180
        self.noise_action = 0;
        self.size = size
        self.state = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt 
        self.SQRT = 0.
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 )
        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b // self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)


    def step(self, dt):

        self.time_elapsed += dt

        self.block[:] = -1
        self.state_col = self.state.copy()
        self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
        self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
        self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))

        sx0 = 0
        sx1 = 0
        for i in range(len(self.state[:, 0])):
            self.lscl[i] = -1
            b = int(math.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * int(math.floor(np.abs(self.state[i, 1])))
            self.lscl[i] = self.block[b]
            self.block[b] = i

        for b0 in range(self.R2):
            f = int(self.block[b0])  
            while f != -1:
                for b1 in range(8):
                    b2 = int(self.cellb[b0][b1])
                    j = int(self.block[b2])  
                    while j != -1:
                        #dist = np.sqrt((self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)


                        #if dist < self.RatioOfInteraction and f != j:
                        sx0 += self.state_orts[j, 2]
                        sx1 += self.state_orts[j, 3]

                        j = int(self.lscl[j])
                        
                suc = 0.5
                self.noise_action = self.noise - 2 * self.noise * np.random.rand()
                
                Vx = self.state[f, 2]
                Vy = self.state[f, 3]
                
                self.state_orts[f, 2] = ((Vx + suc*sx0) * np.cos(self.noise_action) - (Vy + suc*sx1) * np.sin(self.noise_action)) / np.hypot((Vx + suc*sx0), (Vy + suc*sx1))
                self.state_orts[f, 3] = ((Vx + suc*sx0) * np.sin(self.noise_action) + (Vy + suc*sx1) * np.cos(self.noise_action)) / np.hypot((Vx + suc*sx0), (Vy + suc*sx1))
                
                '''
                if (np.abs(sx0) < 0.0001) & (np.abs(sx1) < 0.0001):
                    Vx = self.state[f, 2]
                    Vy = self.state[f, 3]

                    self.state_orts[f, 2] = Vx * np.cos(self.noise_action) - Vy * np.sin(self.noise_action)
                    self.state_orts[f, 3] = Vx * np.sin(self.noise_action) + Vy * np.cos(self.noise_action)
                else:
                    self.state_orts[f, 2] = (sx0 * np.cos(self.noise_action) - sx1 * np.sin(self.noise_action)) / np.hypot(sx0, sx1)
                    self.state_orts[f, 3] = (sx0 * np.sin(self.noise_action) + sx1 * np.cos(self.noise_action)) / np.hypot(sx0, sx1)
                '''
                sx0 = 0
                sx1 = 0
                self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                    self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2))
                    self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                    x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                    y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                    Vx = self.state[f, 2]
                    Vy = self.state[f, 3]

                    self.state[f, 3] = (Vy * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 * Vx * x01 * y01) / self.RadiusSystem ** 2
                    self.state[f, 2] = (Vx * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 * Vy * x01 * y01) / self.RadiusSystem ** 2

                    self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                    self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                    if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                        self.state[f, 0] = x01
                        self.state[f, 1] = y01
    
                else:
                    self.state[f, 0] += dt * self.state[f, 2]
                    self.state[f, 1] += dt * self.state[f, 3]
                            
                f = int(self.lscl[f])
                
            

fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                     xlim=(-RadiusSystem - 1, RadiusSystem + 1), ylim=(-RadiusSystem - 1, RadiusSystem + 1))

particles, = ax.plot([], [], 'bo', ms=1)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

rect = plt.Rectangle(box.bounds[::2],
                     box.bounds[1] - box.bounds[0],
                     box.bounds[3] - box.bounds[2],
                     ec='none', lw=2, fc='none')
ax.add_patch(rect)


def init():
    global box, rect
    particles.set_data([], [])
    rect.set_edgecolor('none')
    time_text.set_text('')
    return particles, rect, time_text


def animate(i):
    global box, rect, dt, ax, fig
    box.step(dt)

    ms = int(fig.dpi * 5 * box.size)

    rect.set_edgecolor('w')
    particles.set_data(box.state[:, 0], box.state[:, 1])
    particles.set_markersize(ms)
    field = plt.quiver(box.state[:, 0], box.state[:, 1],box.state[:, 2], box.state[:, 3],edgecolor='k',width=0.01,scale=400)
    time_text.set_text('time = %.3f' % box.time_elapsed)
    rw = plt.Circle((box.state[0, 0], box.state[0, 1]) , 1. ,fc='r',alpha=0.5)
    ax.add_patch(rw)
    return particles, rect, time_text, field, rw


t0 = time()
animate(0)
t1 = time()
interval = 1000 * dt - (t1 - t0)

ani = animation.FuncAnimation(fig, animate, frames=600,
                              interval=interval, blit=True, init_func=init)
circ = plt.Circle((0, 0), RadiusSystem + 0.1, fc='w')
ax.add_patch(circ)

# ani.save('particle_box_Viscek.wmv', fps=100, extra_args=['-vcodec', 'libx264'])
print("Done!")
plt.show()


RadiusSystem = 6, Number_of_particles = 100, Density = 3, dt = 0.03333333333333333
Done!
